In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.guillaumerozier.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains a script that automatically update data. In the morning it update World data, and it updates French data as soon as they are released by Santé publique France.
"""

'\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.guillaumerozier.fr\nMail : guillaume.rozier@telecomnancy.net\n\nThis file contains a script that automatically update data. In the morning it update World data, and it updates French data as soon as they are released by Santé publique France.\n'

In [7]:
import datetime
import time
import subprocess
import requests

### FUNCTION DEFINITIONS ###
url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"

def update_repo():
    subprocess.run(["sudo", "git", "fetch", "--all"])
    subprocess.run(["sudo", "git", "reset", "--hard", "origin/master"])
    subprocess.run(["sudo", "jupyter", "nbconvert", "--to", "script", "*.ipynb"])
    
def push(type_data):
    subprocess.run(["sudo", "git", "add", "images/", "data/"])
    subprocess.run(["sudo", "git", "commit", "-m", "[auto] data update: {}".format(type_data)])
    subprocess.run(["git", "push"])
    print("pushed")
    
def try_update_france():
    metadata = requests.get(url_metadata)
    data_today = False
    
    if "donnees-hospitalieres-covid19-2020-{}-{}".format(x.strftime("%m"), x.strftime("%d")) in str(metadata.content):
        print("starting France update: {}:{}".format(str(h), str(m)))
        update_repo()

        # Si la data actuelle correspond à celle du fichier
        day_fr_update = d # Alors le fichier est à jour

        # Mise à jour des graphiques
        subprocess.run(["sudo", "python3", "covid19_france_charts.py"])
        push("France")
        print("update France: " + str(h) + ":" + str(m))

        subprocess.run(["sudo", "python3", "covid19_france_maps.py"])
        push("France GIF")
        print("update France GIF: " + str(h) + ":" + str(m))
    return day_fr_update

    
### MAIN FUNCTION ###
x = datetime.datetime.now()
data_today = False
day_world_update = x.strftime("%D")
day_fr_update = x.strftime("%D")
k = 1
print("will enter loop")
while True:
    x = datetime.datetime.now()
    h = x.strftime("%H")
    m = x.strftime("%M")
    d = x.strftime("%D")
    
    
    if (h == '04') & (day_world_update != d):
        print("starting world update: {}:{}".format(str(h), str(m)))
        # S'il est 05h05
        
        day_world_update = d
        
        # mise à jour des graphiques world
        update_repo()
        subprocess.run(["sudo", "python3", "covid19_world_charts.py"])
        push("World")
        print("update World pushed: " + str(h) + ":" + str(m))
        
        # ... et France (certains en dépendent)
        subprocess.run(["sudo", "python3", "covid19_france_charts.py"])
        push("France")
        print("update France pushed: " + str(h) + ":" + str(m))
        time.sleep(30)
        
            
    if ( day_fr_update != d ):
        if (((float(h) == 18) & (float(m) >= 59)) or ((float(h) >= 19) & (float(h) <= 21))):
            while ( (((float(h) == 18) & (float(m) >= 59)) or ((float(h) >= 19) & (float(h) <= 21))) ):
                # Si l'heure comprise entre 18h59 et 21h59, ET les données PAS à jour
                day_fr_update = try_update_france()
                time.sleep(20)
        else:
            if (k%30 == 0): #Check 1 fois toutes les 20 sec * 30 (environ 10 min)
                day_fr_update = try_update_france()
            k += 1
            
    time.sleep(30)

will enter loop


KeyboardInterrupt: 

0